# Convergence Rate for Pure, First-Order Feedback on SO(2) and SO(3)

This is an attempt for simple, empirical / numeric checks for expected convergence rates against:
- feedback and integration as ODE for scalar Euclidean space $\mathbb{R}^1$ \
  (if I can't do this, then ain't no worth moving forward here)
- feedback on $\dot{theta}$ and integration as ODE on $SO(2)$
- feedback on $\omega$ and integration as ODE on $SO(3)$

## Helper Stuff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pydrake.all import (
    LeafSystem,
    Simulator,
    Integrator,
    DiagramBuilder,
    AngleAxis,
    RotationMatrix,
)

In [ ]:
class SimpleVector(LeafSystem):
    def __init__(self, num_q, calc_qd):
        super().__init__()

        self.q = self.DeclareVectorInputPort("q", num_q)

        def calc_qd_sys(context, output):
            q = self.q.Eval(context)
            qd = calc_qd(q)
            output.set_value(qd)

        self.qd = self.DeclareVectorOutputPort("qd", num_q, calc_qd_sys)


def integrate(q0, calc_qd, tf, *, accuracy=1e-5):
    builder = DiagramBuilder()
    integ = builder.AddSystem(Integrator(num_q))
    q_actual_port = integ.get_output_port()
    controller = builder.AddSystem(SimpleVector(num_q, calc_qd))
    builder.Connect(q_actual_port, controller.q)
    builder.Connect(controller.qd, integ.get_input_port())

    diagram = builder.Build()
    diagram_context = diagram.CreateDefaultContext()
    integ_context = integ.GetMyContextFromRoot(diagram_context)
    controller_context = controller.GetMyContextFromRoot(diagram_context)

    integ.set_integral_value(integ_context, q0)

    ts = []
    q_as = []

    def monitor(diagram_context_in):
        assert diagram_context_in is diagram_context
        t = diagram_context.get_time()
        q_actual = q_actual_port.Eval(integ_context)
        ts.append(t)
        q_as.append(q_actual)

    simulator = Simulator(diagram, diagram_context)
    simulator.set_monitor(monitor)
    integrator = simulator.get_mutable_integrator()
    integrator.set_target_accuracy(accuracy)
    try:
        simulator.AdvanceTo(tf)
    except Exception:
        t = diagram_context.get_time()
        print(f"Error at t={t}")
        raise
    return ts, q_as

def maxabs(x):
    return np.max(np.abs(x))

In [ ]:
def rot(th):
    s = np.sin(th)
    c = np.cos(th)
    return np.array([
        [c, -s],
        [s, c],
    ])

def rot_jac(th):
    s = np.sin(th)
    c = np.cos(th)
    return np.array([
        [-s, -c],
        [c, -s],
    ])

def to_th(R):
    x = R[0, 0]
    y = R[1, 0]
    return np.arctan2(y, x)

def rot_err(R_actual, R_des):
    dR = R_actual @ R_des.T
    return to_th(dR)

def flatten(R):
    return R.flat[:]

def unflatten(q):
    n = int(np.sqrt(len(q)))
    assert len(q) == n * n
    return q.reshape((n, n))

def axang(axis, angle):
    return RotationMatrix(AngleAxis(angle, axis)).matrix()

def to_axang3(R):
    axang = AngleAxis(R)
    axang3 = axang.angle() * axang.axis()
    return axang3

def normalize(x, *, tol=1e-10):
    x = np.asarray(x)
    n = np.linalg.norm(x)
    assert n >= tol
    return x / n

def skew(r):
    r1, r2, r3 = r
    return np.array(
        [
            [0, -r3, r2],
            [r3, 0, -r1],
            [-r2, r1, 0],
        ]
    )

def unskew(R, *, tol=1e-10):
    if tol is not None:
        dR = R + R.T
        assert np.all(np.max(np.abs(dR)) < tol)
    r1 = R[2, 1]
    r2 = R[0, 2]
    r3 = R[1, 0]
    return np.array([r1, r2, r3])

def so3_angle(R):
    # same as AngleAxis(R).angle()
    th = np.arccos((np.trace(R) - 1) / 2)
    return th

def so3_dist(R_a, R_b):
    return so3_angle(R_a.T @ R_b)

meh = np.array([[1, 2], [3, 4]])
assert (meh == unflatten(flatten(meh))).all()

## Sanity $\mathbb{R}^1$ Check

Dirt simple, coordinate $x$, with $x_0 \ne 0$, controller and solution should be

\begin{align}
    \dot{x} = -x \\
    x(t) = x_0 e^{-t}
\end{align}

In [ ]:
x0 = 1.0
q0 = [x0]
num_q = len(q0)

def calc_x_expected(t):
    x = x0 * np.exp(-t)
    return x

def q_to_x(q):
    x, = q
    return x

def calc_qd(q):
    x = q_to_x(q)
    x_err = x
    xd = -x_err 
    qd = [xd]
    return qd

ts, q_as = integrate(q0=q0, calc_qd=calc_qd, tf=5.0)
x_as = [q_to_x(q_a) for q_a in q_as]
x_es = [calc_x_expected(t) for t in ts]
ts, x_as, x_es = map(np.asarray, (ts, x_as, x_es))

plt.figure()
plt.plot(ts, x_es, "k--", linewidth=3, label="expected")
plt.plot(ts, x_as, "b-", label="actual")
plt.xlabel("t [s]")
plt.ylabel("x")
plt.legend()

plt.figure()
plt.plot(ts, np.abs(x_as - x_es))
plt.xlabel("t [s]")
plt.ylabel("Error")
pass

## SO(2)

General rotation:

\begin{align}
    c = cos(\theta) \\
    s = sin(\theta) \\
    R(\theta) =
        \begin{bmatrix}
            c & -s \\
            s & c
        \end{bmatrix} \\
    \dot{R}(\theta) =
        \dot{\theta} \begin{bmatrix}
            -s & -c \\
            c & -s
        \end{bmatrix}
\end{align}

Define simple first-order feedback and expected closed-form solution:

\begin{align}
    \dot{\theta} = -\theta \\
    \theta(t) = \theta_0 e^{-t}
\end{align}

But for kicks and giggles, integrate via coordinate change as $R(t) = \int_0^t \dot{R}\left(\theta(R), \dot{\theta}\right) \ dt$

In [ ]:
theta0 = 2.0
R0 = rot(theta0)
q0 = flatten(R0)
num_q = len(q0)

def calc_th_expected(t):
    theta = theta0 * np.exp(-t)
    return theta

def q_to_th(q):
    R = unflatten(q)
    so2_tol = 1e-5
    so2_err = R @ R.T - np.eye(2)
    assert maxabs(so2_err) < so2_tol, so2_err
    return to_th(R)

def calc_qd(q):
    R = unflatten(q)
    th = to_th(R)
    th_err = th
    thd = -th_err
    Rd = rot_jac(th) * thd
    qd = flatten(Rd)
    return qd

ts, q_as = integrate(q0=q0, calc_qd=calc_qd, tf=5.0)
th_as = [q_to_th(q_a) for q_a in q_as]
th_es = [calc_th_expected(t) for t in ts]
ts, th_as, th_es = map(np.asarray, (ts, th_as, th_es))

plt.figure()
plt.plot(ts, th_es, "k--", linewidth=3, label="expected")
plt.plot(ts, th_as, "b-", label="actual")
plt.xlabel("t [s]")
plt.ylabel("th [rad]")
plt.legend()

plt.figure()
plt.plot(ts, np.abs(th_as - th_es), "g")
plt.xlabel("t [s]")
plt.ylabel("Error [rad]")
pass

## SO(3)

FWIW: The following has convergence *bounds* regarding convergence of (second-order) feedback linearization on $SE(3)$:

[1] Lee, Taeyoung, Melvin Leok, and N. Harris McClamroch. \
&nbsp;&nbsp; “Control of Complex Maneuvers for a Quadrotor UAV Using Geometric Methods on SE(3).” arXiv, March 9, 2010. \
&nbsp;&nbsp; https://doi.org/10.48550/arXiv.1003.2005.

But would be nice to have an explicit solution for first-order feedback for both $SO(2)$ and $SO(3)$.

Use formulation as shown in [1], Eq. (10), but change order of multiplication so that we're computing the derivative
to align with the *world frame* angular velocity, not the *body frame* angular velocity.

Per that paper, let:

- $\hat{}: \mathbb{R}^3 \rightarrow so(3)$ be the operator produces skew matrices,
- $^\vee: so(3) \rightarrow \mathbb{R}^3$ be its inverse

First, restate the standard relationship between $R \in \mathbb{R}^{3 \times 3}$, its time derivatives,
and *world frame* angular velocities:

\begin{align}
    \dot{R} = {}^W\hat{\omega} R \\
    {}^W\hat{\omega} = \dot{R} R^T \\
\end{align}

Then we can get the following position-to-velocity/acceleration feedback:

\begin{align}
    R_\Delta &= R_a R_d^T \\
    \hat{e}_R &= \frac{1}{2} \left( R_\Delta - R_\Delta ^T \right)
\end{align}

Let $R(\lambda, \theta)$ provide us with angle-axis coordinates for rotation, $(\lambda, \theta)$.

First, let $R_0 = R(\lambda, \theta_0)$, for (what is hopefully a) constant $\lambda$ across all time.
Let $R_a = R$, and $R_d = I$.

Using this, we define the following first-order feedback and dynamics, with the expected (?!) solution:

\begin{align}
    ^{W}\omega &= -e_R \\
    \dot{R} &= {}^W\hat{\omega} R \\
    \theta(t) &= \theta_0 e^{-t} \\
    R(t) &= \left[ e^{-\hat{\lambda} \theta_0} \right] ^{-t} \\
        &= R_0 e^{-\hat{\lambda} \theta(t)} \hspace{10pt} \text{er is this even right?!}
\end{align}

Below, we plot the error / geodesic on $SO(3)$ as $d(R, I)$.

**TODO**: Er, the above seems bad, but just doing so(3) based feedback seems great?

In [ ]:
# theta0 = np.pi - 0.1
theta0 = 0.2 # 1.0
# axis = np.array([0, 0, 1])
axis = normalize([0.2, 0.3, 0.4])
R0 = axang(axis, theta0)
q0 = flatten(R0)
num_q = len(q0)

R_d = np.eye(3)

def calc_R_expected(t):
    theta = theta0 * np.exp(-t)
    R = axang(axis, theta)
    return R

def q_to_R(q, *, check=True):
    R = unflatten(q)
    if check:
        so3_tol = 1e-4
        so3_err = R @ R.T - np.eye(3)
        assert maxabs(so3_err) < so3_tol, so3_err
    return R

def rot_error_paper(R_a, R_d):
    dR = R_a @ R_d.T
    err_hat = (dR - dR.T) / 2
    return err_hat

def rot_error_axang(R_a, R_d):
    R = R_a @ R_d.T
    axang3 = to_axang3(R)
    return skew(axang3)

def calc_qd(q):
    R = q_to_R(q, check=False)
    err_hat = rot_error_paper(R, R_d)
    # err_hat = rot_error_axang(R, R_d)
    w_hat = -err_hat
    Rd = w_hat @ R
    qd_actual = flatten(Rd)
    return qd_actual

ts, q_as = integrate(q0=q0, calc_qd=calc_qd, tf=5.0, accuracy=1e-6)
R_as = [q_to_R(q_a) for q_a in q_as]
R_es = [calc_R_expected(t) for t in ts]
errs = [so3_dist(R_a, R_e) for (R_a, R_e) in zip(R_as, R_es)]
th_as = [so3_angle(R) for R in R_as]
th_es = [so3_angle(R) for R in R_es]

ts, errs, th_as, th_es = map(np.asarray, (ts, errs, th_as, th_es))

plt.figure()
plt.plot(ts, th_es, "k--", linewidth=3, label="expected")
plt.plot(ts, th_as, "b-", label="actual")
plt.xlabel("t [s]")
plt.ylabel("Distance from Desired: $d(R_a, R_d)$ [rad]")
plt.legend()

plt.figure()
plt.plot(ts, errs, "g")
plt.xlabel("t [s]")
plt.ylabel("Distance from Expected: $d(R_a, R_{a,e})$ [rad]")
pass